## Import all what we need

In [1]:
import os
import math
import glob
import requests
import numpy as np
import pandas as pd
from io import StringIO
from tqdm import tqdm_notebook as tqdm

# Part 1: HITRAN Online Information

Get the names of molecules, iso-slugs and isotopoluge datasets from the api__urls.txt which saved the URLs with molecule, iso-slug and isotopologue. Combine them with '/' for reading files from folders more convenient later.

In [2]:
molecule = []
iso_slug = []
isotopologue = []
path_mol_iso = []
for line in open('./data/url/api__urls.txt'):
    molecule.append(line.split('/')[-4])
    iso_slug.append(line.split('/')[-3])
    isotopologue.append(line.split('/')[-2])
    path_mol_iso.append(line.split('/')[-4] + '/' + line.split('/')[-3]
                        + '/' + line.split('/')[-2])

molecule_list = list(set(molecule))
molecule_list.sort(key=molecule.index)

iso_slug_list = list(set(iso_slug))
iso_slug_list.sort(key=iso_slug.index)

isotopologue_list = list(set(isotopologue))
isotopologue_list.sort(key=isotopologue.index)

path_mol_iso_list = list(set(path_mol_iso))
path_mol_iso_list.sort(key=path_mol_iso.index)

print('Molecule:', molecule_list)
print('Iso-slug:', iso_slug_list)
print('Isotopologue:', isotopologue_list)
print('Total:', path_mol_iso_list)


Molecule: ['AlH', 'C2H2', 'C2', 'CO2', 'H2O', 'H3O_p', 'NH3', 'TiO']
Iso-slug: ['27Al-1H', '12C2-1H2', '12C2', '12C-16O2', '1H2-16O', '1H3-16O_p', '14N-1H3', '46Ti-16O', '47Ti-16O', '48Ti-16O', '49Ti-16O', '50Ti-16O']
Isotopologue: ['AlHambra', 'aCeTY', '8states', 'UCL-4000', 'POKAZATEL', 'eXeL', 'CoYuTe', 'Toto']
Total: ['AlH/27Al-1H/AlHambra', 'C2H2/12C2-1H2/aCeTY', 'C2/12C2/8states', 'CO2/12C-16O2/UCL-4000', 'H2O/1H2-16O/POKAZATEL', 'H3O_p/1H3-16O_p/eXeL', 'NH3/14N-1H3/CoYuTe', 'TiO/46Ti-16O/Toto', 'TiO/47Ti-16O/Toto', 'TiO/48Ti-16O/Toto', 'TiO/49Ti-16O/Toto', 'TiO/50Ti-16O/Toto']


Convert the iso-slug names into the ones which are shown in the table of HITRAN online website. It will help us to get their corresponding molecule numbers, isotopologue numbers and fractional abundances. 

The HITRAN online URL is: https://hitran.org/docs/iso-meta/.

In [3]:
unc_formula = pd.DataFrame(eval(str(iso_slug_list).replace('1H','H')
                                .replace('-','').replace('_p','+')))
unc_formula.columns = ['exomol formula']
unc_formula

,exomol formula
0,27AlH
1,12C2H2
2,12C2
3,12C16O2
4,H216O
5,H316O+
6,14NH3
7,46Ti16O
8,47Ti16O
9,48Ti16O


Set $C_2$ information for HITRAN format since this molecule is not listed in HITRAN online table. We set its molecule number to be 51 and its isotopolugue number to be 1. Set its fractional abundances of iso-slug to be 1 as well.

In [4]:
path_mol_iso_list = ['C2/12C2/8states']
hitran_online = pd.DataFrame()
hitran_online['molecule ID'] = ['51']
hitran_online['isotopologue ID'] = ['1']
hitran_online['exomol formula'] = ['12C2']
hitran_online['fractional abundance'] = ['1']
hitran_online


,molecule ID,isotopologue ID,exomol formula,fractional abundance
0,51,1,12C2,1


In [5]:
path_mol_iso = path_mol_iso_list[0]
M_mol_iso = 'M of ' + path_mol_iso.replace('/','__')
molecule_id = int(hitran_online['molecule ID'][0])
isotopologue_id = int(hitran_online['isotopologue ID'][0])
fractional_abundance = float(hitran_online['fractional abundance'][0])

In [6]:
read_path = './data/www.exomol.com/db/'

# Create a folder for saving result files.
# If the folder exists, save files directory,otherwise, create it.
result_path = './data/result/csv/'
# Determine whether the folder exists or not.
if os.path.exists(result_path):
    pass
else:
    # Create the folder.
    os.makedirs(result_path, exist_ok=True)

# Part 2: Process Data

## 2.1 Read States File

Consider column names of states file with def files.

In [7]:
path_mol_iso = path_mol_iso_list[0]
def_path = glob.glob('./data/def/' + '*' + path_mol_iso.split('/')[1]
                     + '__' + path_mol_iso.split('/')[2] + '.def')
def_reader = pd.read_csv(def_path[0], sep='\\s+', names=['1','2','3','4','5'], header=None)
list(def_reader[def_reader['4'].isin(['label'])]['1'].values)

['+/-', 'e/f', 'State', 'v', 'Lambda', 'Sigma', 'Omega']

In [8]:
states_col_name = (['ID','energy','g_tot','J_tot','Unc']
                   + ['tau','g-fac','+/-','e/f','State','v','Lambda',
                      'Sigma','Omega','F','N','d/m','Ecalc'])

Read compressed states file in chunks directly. Extract rows of states file whose uncertainty indices are small than 0.001 to be the considered states file.

In [10]:
s_df = dict()
states_df = pd.DataFrame()
states_filenames = glob.glob(read_path + path_mol_iso + '/' + path_mol_iso.split('/')[1]
                             + '__' + path_mol_iso.split('/')[2] + '.states.bz2')

for states_filename in states_filenames:
    s_df[states_filename] = pd.read_csv(states_filename, compression='bz2', sep='\s+',
                                        header=None, names=states_col_name,
                                        chunksize=100_000_000, iterator=True,
                                        low_memory=False)
    for chunk in s_df[states_filename]:
        states_df = states_df.append(chunk)
        
# Extract rows of states file whose uncertainty indices are small than 0.001.
unc_states_df = states_df[states_df['Unc'] < float(0.001)]
pd.set_option("display.max_columns",30)
unc_states_df

,ID,energy,g_tot,J_tot,Unc,tau,g-fac,+/-,e/f,State,v,Lambda,Sigma,Omega,F,N,d/m,Ecalc
1,2,1827.487599,1,0,0.0009,1705.800000,0.0000,+,e,X1Sigmag+,1,0,0,0,F1,0,m,1827.480518
7,8,8844.125829,1,0,0.0008,0.194230,0.0000,+,e,X1Sigmag+,5,0,0,0,F1,0,m,8843.929075
12,13,12154.962083,1,0,0.0006,1.662900,0.0000,+,e,X1Sigmag+,7,0,0,0,F1,0,m,12154.991788
17,18,15302.893734,1,0,0.0007,0.283140,0.0000,+,e,X1Sigmag+,9,0,0,0,F1,0,m,15301.094704
31,32,20974.503876,1,0,0.0000,0.000006,0.0000,+,e,b3Sigmag-,11,0,0,0,F3,1,d,20974.503876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42998,42999,48323.939883,307,153,0.0000,NaN,0.0001,+,f,b3Sigmag-,11,0,1,1,F2,153,d,48323.939883
43085,43086,48323.758459,309,154,0.0000,NaN,0.0130,+,e,b3Sigmag-,11,0,1,1,F1,153,d,48323.758459
43089,43090,48942.551552,309,154,0.0000,NaN,-0.0129,+,e,b3Sigmag-,11,0,0,0,F3,155,d,48942.551552
43153,43154,48942.827037,311,155,0.0000,NaN,0.0001,+,f,b3Sigmag-,11,0,1,1,F2,155,d,48942.827037


## 2.2 Read Partition Function File

In [11]:
pf_col_name = ['T', 'Q']
pf_path = read_path + path_mol_iso + '/' + path_mol_iso.replace('/','__') + '_pf.csv'

pf_url = ('http://www.exomol.com/db/' + path_mol_iso + '/'
          + path_mol_iso.split('/')[1] + '__' + path_mol_iso.split('/')[2] + '.pf')   
response = requests.get(pf_url)
content = response.text  
pf_data = pd.read_csv(StringIO(content), sep='\s+', names=pf_col_name, header=None, engine='python')
pf_data.to_csv(pf_path, header=False)

pf_df = pd.read_csv(pf_path, header=None, names=pf_col_name)
# Partition function defined as sum over states at standard 296K
Q = pf_df.iloc[296-1]['Q']
Q

76.2329

## 2.3 Read Transitions Files

Extract rows of transitionos files whose upper states ID and lower states ID are all in considered states file.

In [13]:
def extract_unc_trans(trans_df):
    upper_id = trans_df['i'].values
    lower_id = trans_df['f'].values
    state_id = unc_states_df['ID'].values
    
    # Extract the same upper states ID from states_df
    unc_trans_i_df = pd.DataFrame()
    for id_1 in tqdm(state_id):
        unc_trans_i_df = unc_trans_i_df.append(trans_df[trans_df['i'].isin([id_1])])
        
    # Extract the same lower states ID from states_df
    unc_trans_df = pd.DataFrame()
    for id_2 in tqdm(state_id):
        unc_trans_df = unc_trans_df.append(unc_trans_i_df[unc_trans_i_df['f'].isin([id_2])])
        
    return unc_trans_df

## 2.4 Calculating

HITRAN Parameters for calculating.

In [12]:
T = 296                      # Reference temperature k
h = 6.62607015e-34           # Planck's const (J s)
c = 299792458                # Velocity of light (m s-1)
kB = 1.380649e-23            # Boltzmann's const (J K-1)

c2 = h * c * 100 / kB                  # Second radiation constant (cm K)
pi_c_8 = 1 / (8 * np.pi * c * 100)     # 8 * pi * c (cm-1 s)
c2_T = c2 / T                          # c2 / T  (cm)


Process data for CSV format.

In [14]:
def calculate_csv(unc_states_df, unc_trans_df):
    unc_upper_id = unc_trans_df['i'].values
    unc_lower_id = unc_trans_df['f'].values 
    state_id = unc_states_df['ID']
    unc_trans_num = unc_trans_df['i'].count()

    wavenumber = []                         # Vacuum wavenumber (cm−1)
    intensity = pd.DataFrame()              # Intensities (cm-1/molecule cm-2) at standard 296K         
    A_coefficient = []                      # Einstein A-coefficient
    lower_state_energy = pd.DataFrame()     # lower state energy
    uncertainty = []                        # Uncertainty indices
    weight_upper_state = pd.DataFrame()     # Statistical weight of upper state
    weight_lower_state = pd.DataFrame()     # Statistical weight of lower state
    upper_global_quanta = []                # Upper-state 'global' quanta
    lower_global_quanta = []                # Lower-state 'global' quanta
    upper_local_quanta = []                 # Upper-state 'local' quanta
    lower_local_quanta = []                 # Lower-state 'local' quanta

    for i in tqdm(range(unc_trans_num)):
        id_i = unc_upper_id[i]
        id_f = unc_lower_id[i]
        A = unc_trans_df['A_if'].values[i]                               # Einstein-A coefficient (s−1)
        g_i = unc_states_df[state_id.isin([id_i])]['g_tot'].values       # Total degeneracy of upper state
        g_f = unc_states_df[state_id.isin([id_f])]['g_tot'].values       # Total degeneracy of lower state
        E_i = unc_states_df[state_id.isin([id_i])]['energy'].values      # Upper state energy
        E_f = unc_states_df[state_id.isin([id_f])]['energy'].values      # Lower state energy
        unc_i = unc_states_df[state_id.isin([id_i])]['Unc'].values       # Uncertainty indices of upper state
        unc_f = unc_states_df[state_id.isin([id_f])]['Unc'].values       # Uncertainty indices of lower state
        
        u_State = unc_states_df[state_id.isin([id_i])]['State'].values[0]
        u_v = unc_states_df[state_id.isin([id_i])]['v'].values[0]
        u_Omega = unc_states_df[state_id.isin([id_i])]['Omega'].values[0]
        u_F = unc_states_df[state_id.isin([id_i])]['F'].values[0]
        u_N = unc_states_df[state_id.isin([id_i])]['N'].values[0]
        u_J = unc_states_df[state_id.isin([id_i])]['J_tot'].values[0]
        u_ef = unc_states_df[state_id.isin([id_i])]['e/f'].values[0]

        l_State = unc_states_df[state_id.isin([id_f])]['State'].values[0]
        l_v = unc_states_df[state_id.isin([id_f])]['v'].values[0]
        l_Omega = unc_states_df[state_id.isin([id_f])]['Omega'].values[0]
        l_F = unc_states_df[state_id.isin([id_f])]['F'].values[0]
        l_N = unc_states_df[state_id.isin([id_f])]['N'].values[0]
        l_J = unc_states_df[state_id.isin([id_f])]['J_tot'].values[0]
        l_ef = unc_states_df[state_id.isin([id_f])]['e/f'].values[0]
        
        V_i = '%9s%2d%2d%2s%3d' % (u_State,u_v,u_Omega,u_F,u_N) + ','    # Upper-state 'global' quanta
        V_f = '%9s%2d%2d%2s%3d' % (l_State,l_v,l_Omega,l_F,l_N) + ','    # Lower-state 'global' quanta
        Q_i = '       %3d%2s' % (u_J,u_ef) + ','                         # Upper-state 'local' quanta
        Q_f = '       %3d%2s' % (l_J,l_ef) + ','                         # Lower-state 'local' quanta

        unc = math.sqrt(unc_i ** 2 + unc_f ** 2)                         # Uncertainty idices
        v = float(abs(E_i - E_f))                                        # Vacuum wavenumber (cm−1)
        S = g_i * A * np.exp(- c2_T * E_f) * (1 - np.exp(- c2_T * v)) * pi_c_8 / (v ** 2) / Q    # Intensities

        wavenumber.append(v)
        intensity = intensity.append(pd.DataFrame(S))
        A_coefficient.append(A)
        lower_state_energy = lower_state_energy.append(pd.DataFrame(E_f))
        uncertainty.append(unc)
        weight_upper_state = weight_upper_state.append(pd.DataFrame(g_i))
        weight_lower_state = weight_lower_state.append(pd.DataFrame(g_f))
        upper_global_quanta += V_i.split(',')
        lower_global_quanta += V_f.split(',')
        upper_local_quanta += Q_i.split(',')
        lower_local_quanta += Q_f.split(',')
    
    iso_csv_df = pd.DataFrame()
    iso_csv_df['v'] = wavenumber                       # Vacuum wavenumber (cm−1)
    iso_csv_df['S'] = intensity.values                 # Intensities (cm-1/molecule cm-2) at standard 296K  
    iso_csv_df['A'] = A_coefficient                    # Einstein A-coefficient
    iso_csv_df['E_f'] = lower_state_energy.values      # Lower state energy
    iso_csv_df['Ierr'] = uncertainty                   # Uncertainty indices
    iso_csv_df['g_i'] = weight_upper_state.values      # Statistical weight of upper state
    iso_csv_df['g_f'] = weight_lower_state.values      # Statistical weight of lower state
    iso_csv_df[M_mol_iso] = molecule_id                # Molecule number
    iso_csv_df['I'] = isotopologue_id                  # Isotopologue number
    iso_csv_df['gm_a'] = np.nan                        # Air-broadened half-width
    iso_csv_df['gm_s'] = np.nan                        # Self-broadened half-width
    iso_csv_df['n_a'] = np.nan                         # Temperature-dependence exponent for gamma_air
    iso_csv_df['dt_a'] = np.nan                        # Air pressure-induced line shift
    iso_csv_df['V_i'] = list(filter(None, upper_global_quanta))            # Upper-state 'global' quanta
    iso_csv_df['V_f'] = list(filter(None, lower_global_quanta))            # Lower-state 'global' quanta
    iso_csv_df['Q_i'] = list(filter(None, upper_local_quanta))             # Upper-state 'local' quanta
    iso_csv_df['Q_f'] = list(filter(None, lower_local_quanta))             # Lower-state 'local' quanta                  
    iso_csv_df['Iref'] = np.nan                        # Reference indices
    iso_csv_df['*'] = np.nan                           # Flag

    return iso_csv_df
    

# Part 3: Save as CSV Format

In [15]:
trans_col_name = ['i', 'f', 'A_if']
t_df = dict()
species_csv_df = pd.DataFrame()
trans_filenames = glob.glob(read_path + path_mol_iso + '/' + '*trans.bz2')
for trans_filename in tqdm(trans_filenames):
    t_df[trans_filename] = pd.read_csv(trans_filename, compression='bz2', sep='\s+',
                                       usecols=[0,1,2], header=None, names=trans_col_name,
                                       chunksize=100_000_000, iterator=True, low_memory=False)
    # Set an empty DataFrame to avoid meeting empty considered transitions files.
    iso_csv_df = pd.DataFrame()
    for trans_df in t_df[trans_filename]:
        unc_trans_df = extract_unc_trans(trans_df)
        if len(unc_trans_df) != 0:
            iso_csv_df = calculate_csv(unc_states_df, unc_trans_df)
            
    species_csv_df = species_csv_df.append(iso_csv_df)

order = [M_mol_iso, 'I', 'v', 'S', 'A', 'gm_a', 'gm_s', 'E_f', 'n_a', 'dt_a',
         'V_i', 'V_f', 'Q_i', 'Q_f', 'Ierr', 'Iref', '*', 'g_i', 'g_f']
species_csv_df = species_csv_df[order]
# Sort by increasing wavenumber
species_csv_df = species_csv_df.sort_values(['v'], ascending = True).reset_index(drop=True)
# Save into a CSV file with column names which contain molecule name.
species_csv_df.to_csv(result_path + path_mol_iso.replace('/','__') + '.csv', header=True, index=False) 

In [16]:
species_csv_df

,M of C2__12C2__8states,I,v,S,A,gm_a,gm_s,E_f,n_a,dt_a,V_i,V_f,Q_i,Q_f,Ierr,Iref,*,g_i,g_f
0,51,1,0.763159,7.226895e-59,3.592000e-09,NaN,NaN,16783.911529,NaN,NaN,B1Deltag 3 2F1 24,A1Piu 5-1F1 23,24 e,23 e,0.001140,NaN,NaN,49,47
1,51,1,0.862149,9.499103e-56,8.062300e-09,NaN,NaN,15455.953238,NaN,NaN,A1Piu 4-1F1 25,B1Deltag 2 2F1 24,25 e,24 e,0.001140,NaN,NaN,51,49
2,51,1,5.170811,1.029798e-55,3.234400e-06,NaN,NaN,15986.308668,NaN,NaN,A1Piu 5-1F1 5,B1Deltag 3 2F1 5,5 e,5 f,0.001131,NaN,NaN,11,11
3,51,1,5.633179,3.142049e-62,8.716300e-14,NaN,NaN,15306.994134,NaN,NaN,X1Sigmag+ 9 0F1 2,c3Sigmau+ 3 0F3 3,2 e,2 f,0.000707,NaN,NaN,5,5
4,51,1,6.029034,7.323290e-62,2.172200e-13,NaN,NaN,15306.598279,NaN,NaN,X1Sigmag+ 9 0F1 2,c3Sigmau+ 3-1F2 3,2 e,3 e,0.000707,NaN,NaN,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46751,51,1,38841.250637,2.119960e-32,2.877900e-09,NaN,NaN,675.231937,NaN,NaN,b3Sigmag-29 0F3 9,a3Piu 0-2F1 6,8 e,7 e,0.000900,NaN,NaN,17,15
46752,51,1,38843.599225,1.478294e-31,2.186800e-08,NaN,NaN,637.686609,NaN,NaN,b3Sigmag-29 0F3 7,a3Piu 0-2F1 4,6 e,5 e,0.000900,NaN,NaN,13,11
46753,51,1,38845.229459,3.183554e-31,6.009900e-08,NaN,NaN,612.186475,NaN,NaN,b3Sigmag-29 0F3 5,a3Piu 0-2F1 2,4 e,3 e,0.000900,NaN,NaN,9,7
46754,51,1,38909.357394,1.170167e-45,7.216400e-07,NaN,NaN,8526.798394,NaN,NaN,b3Sigmag-29 1F1 69,a3Piu 0 2F1 70,70 e,71 e,0.000000,NaN,NaN,141,143
